In [1]:
from arcgis.gis import GIS
gis = GIS("home")

In [2]:
%matplotlib inline
#ArcGIS packages
import arcpy
#from arcgis.mapping import WebScene
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from IPython.display import display

from arcgis.features import GeoAccessor
from arcgis import *
from arcpy.sa import Raster, Int  # Raster float to integer
# Raster processing for dataframe
from rasterstats import zonal_stats

# basic packages
import csv
import numpy as np
import os
import timeit
import random
import string
from pprint import pprint
import math

# Data management
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point  # to get points from long lat
import gc
import shutil
import tempfile
import ast
from collections import Counter
from typing import Optional, List

from sklearn.neighbors import BallTree

# Request service
#from requests import Request
import json
import re
from functools import reduce
#from owslib.wfs import WebFeatureService

# Plotting packages
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
import multiprocessing
from collections import Counter

In [5]:
merged_df_final = pd.read_csv(r"D:\NDIS_Database\13_NDIS_Display\Page2Layers\NDIS380_displayonly.csv")
merged_df_final.info()

C:/Users/Dell/AppData/Local/Temp/ArcGISProTemp15124/xpython_15124/3707028055.py:1: DtypeWarning: Columns (12,13,14,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df_final = pd.read_csv(r"D:\NDIS_Database\13_NDIS_Display\Page2Layers\NDIS380_displayonly.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1931885 entries, 0 to 1931884
Data columns (total 22 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   HazardID               int64  
 1   latitude               float64
 2   longitude              float64
 3   HazardType             object 
 4   distance               float64
 5   pop                    float64
 6   intensity              float64
 7   economic_loss_million  float64
 8   duration_minutes       float64
 9   travel_time            float64
 10  monitor_time           float64
 11  cpm_total_time         float64
 12  DroneModel1            object 
 13  DroneModel2            object 
 14  DroneModel3            object 
 15  FlightPath_m1          float64
 16  FlightPath_m2          float64
 17  FlightPath_m3          float64
 18  Sensor1                object 
 19  Sensor2                object 
 20  Sensor3                object 
 21  Stage_1                object 
dtypes: float64(13), in

In [158]:
# Select only the desired columns from merged_df_final
merged_trimmed = merged_df_final[[
    "HazardID",
    "latitude",
    "longitude",
    "HazardType",
    "distance",
    "pop",
    "intensity",
    "economic_loss_million",
    "duration_minutes",
    "travel_time",
    "cpm_total_time"
]].copy()
merged_trimmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1931885 entries, 0 to 1931884
Data columns (total 11 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   HazardID               int64  
 1   latitude               float64
 2   longitude              float64
 3   HazardType             object 
 4   distance               float64
 5   pop                    float64
 6   intensity              float64
 7   economic_loss_million  float64
 8   duration_minutes       float64
 9   travel_time            float64
 10  cpm_total_time         float64
dtypes: float64(9), int64(1), object(1)
memory usage: 162.1+ MB


In [159]:
landslide = merged_trimmed[(merged_trimmed["HazardType"] == "Landslide")]
landslide.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14421 entries, 1044 to 1816642
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   HazardID               14421 non-null  int64  
 1   latitude               14421 non-null  float64
 2   longitude              14421 non-null  float64
 3   HazardType             14421 non-null  object 
 4   distance               14064 non-null  float64
 5   pop                    14064 non-null  float64
 6   intensity              14064 non-null  float64
 7   economic_loss_million  14064 non-null  float64
 8   duration_minutes       14064 non-null  float64
 9   travel_time            14063 non-null  float64
 10  cpm_total_time         14063 non-null  float64
dtypes: float64(9), int64(1), object(1)
memory usage: 1.3+ MB


In [160]:
landslide.to_csv(r"D:\NDIS_Database\13_NDIS_Display\Page2Layers\ghz_types\landslide.csv", index=False)

In [161]:
tsunami = merged_trimmed[(merged_trimmed["HazardType"] == "Tsunami")]
tsunami.info()

<class 'pandas.core.frame.DataFrame'>
Index: 738 entries, 15108 to 1817011
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   HazardID               738 non-null    int64  
 1   latitude               738 non-null    float64
 2   longitude              738 non-null    float64
 3   HazardType             738 non-null    object 
 4   distance               369 non-null    float64
 5   pop                    369 non-null    float64
 6   intensity              369 non-null    float64
 7   economic_loss_million  369 non-null    float64
 8   duration_minutes       369 non-null    float64
 9   travel_time            369 non-null    float64
 10  cpm_total_time         369 non-null    float64
dtypes: float64(9), int64(1), object(1)
memory usage: 69.2+ KB


In [162]:
tsunami.to_csv(r"D:\NDIS_Database\13_NDIS_Display\Page2Layers\ghz_types\tsunami.csv", index=False)

In [163]:
nuclear = merged_trimmed[(merged_trimmed["HazardType"] == "Nuclear")]
nuclear.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1541 entries, 1814699 to 1816239
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   HazardID               1541 non-null   int64  
 1   latitude               1541 non-null   float64
 2   longitude              1541 non-null   float64
 3   HazardType             1541 non-null   object 
 4   distance               1541 non-null   float64
 5   pop                    1541 non-null   float64
 6   intensity              1541 non-null   float64
 7   economic_loss_million  1541 non-null   float64
 8   duration_minutes       1541 non-null   float64
 9   travel_time            1541 non-null   float64
 10  cpm_total_time         1541 non-null   float64
dtypes: float64(9), int64(1), object(1)
memory usage: 144.5+ KB


In [164]:
nuclear.to_csv(r"D:\NDIS_Database\13_NDIS_Display\Page2Layers\ghz_types\nuclear.csv", index=False)

# <font color='red'> 1. Volcano data </font>

In [117]:
volcano_df = pd.read_csv(r"D:\NDIS_Database\01_Volcano\eruption_history.csv")
volcano_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11282 entries, 0 to 11281
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Volcano Number            11282 non-null  int64  
 1   Volcano Name              11282 non-null  object 
 2   Eruption Number           11282 non-null  int64  
 3   Eruption Category         11282 non-null  object 
 4   Area of Activity          4845 non-null   object 
 5   VEI                       8343 non-null   float64
 6   VEI Modifier              1307 non-null   object 
 7   Start Year Modifier       1604 non-null   object 
 8   Start Year                11282 non-null  int64  
 9   Start Year Uncertainty    2107 non-null   float64
 10  Start Month               11041 non-null  float64
 11  Start Day Modifier        412 non-null    object 
 12  Start Day                 11037 non-null  float64
 13  Start Day Uncertainty     858 non-null    float64
 14  Eviden

In [118]:
volcano_df

,Volcano Number,Volcano Name,Eruption Number,Eruption Category,Area of Activity,VEI,VEI Modifier,Start Year Modifier,Start Year,Start Year Uncertainty,...,Evidence Method (dating),End Year Modifier,End Year,End Year Uncertainty,End Month,End Day Modifier,End Day,End Day Uncertainty,Latitude,Longitude
0,353010,Fernandina,22560,Confirmed Eruption,SE flank,1.0,NaN,NaN,2024,NaN,...,Observations: Satellite (visual),>,2024.0,NaN,3.0,NaN,4.0,NaN,-0.370,-91.550
1,257040,Ambrym,22554,Confirmed Eruption,NaN,NaN,NaN,NaN,2024,NaN,...,Observations: Reported,NaN,2024.0,NaN,1.0,NaN,16.0,NaN,-16.250,168.120
2,284141,Ahyi,22557,Confirmed Eruption,NaN,NaN,NaN,NaN,2024,NaN,...,Observations: Satellite (visual),>,2024.0,NaN,3.0,NaN,1.0,NaN,20.420,145.030
3,264180,Lewotobi,22556,Confirmed Eruption,NaN,NaN,NaN,NaN,2023,NaN,...,Observations: Reported,>,2024.0,NaN,3.0,NaN,1.0,NaN,-8.542,122.775
4,311110,Kanaga,22558,Confirmed Eruption,NaN,NaN,NaN,NaN,2023,NaN,...,Observations: Reported,NaN,2023.0,NaN,12.0,NaN,18.0,NaN,51.923,-177.168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11277,357121,Quetrupillan,22351,Confirmed Eruption,NaN,3.0,NaN,NaN,-10658,29.0,...,Isotopic: 14C (calibrated),NaN,NaN,NaN,NaN,NaN,NaN,NaN,-39.496,-71.722
11278,221270,Aluto,22531,Confirmed Eruption,Tephra sample from Deka Wede (W of caldera),NaN,NaN,NaN,-10900,1630.0,...,Isotopic: 14C (calibrated),NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.770,38.780
11279,343150,"Yojoa, Lago",22387,Confirmed Eruption,NaN,NaN,NaN,NaN,-11073,173.0,...,Isotopic: 14C (calibrated),NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.964,-87.983
11280,357121,Quetrupillan,22352,Confirmed Eruption,NaN,3.0,NaN,NaN,-11345,932.0,...,Isotopic: 14C (calibrated),NaN,NaN,NaN,NaN,NaN,NaN,NaN,-39.496,-71.722


In [57]:
volcano_df.VEI.unique()

array([ 1., nan,  0.,  2.,  3.,  5.,  4.,  6.,  7.])

In [19]:
volcano_df['Eruption Category'].unique()

array(['Confirmed Eruption', 'Uncertain Eruption', 'Discredited Eruption'],
      dtype=object)

In [115]:
volc.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   OBJECTID          1324 non-null   int64   
 1   Shape             1324 non-null   object  
 2   GmlID             1324 non-null   object  
 3   VolcanoNumber     1324 non-null   int64   
 4   VolcanoName       1324 non-null   object  
 5   Country           1324 non-null   object  
 6   Remarks           1324 non-null   object  
 7   VolcanoType       1324 non-null   object  
 8   LastEruption      857 non-null    float64 
 9   Elevation         1324 non-null   int64   
 10  TectonicSetting   1319 non-null   object  
 11  Within_5km        1291 non-null   float64 
 12  Within_10km       1291 non-null   float64 
 13  Within_30km       1291 non-null   float64 
 14  Within_100km      1291 non-null   float64 
 15  VPImageFileName   1252 non-null   object  
 16  VPImageCaption  

In [116]:
volc.LastEruption.unique()

array([ -8300.,  -4040.,     nan,  -3600.,   1538.,   1944.,  -8040.,
         1302.,   1230.,   2023.,   1890.,   1891.,   1867.,   -258.,
          140.,   1950.,   1888.,  -2080.,  -6880.,   1650.,   1855.,
         1840.,     50.,   -750.,  -1900.,  -2000.,   -778.,  -3000.,
         2008.,   2013.,   2011.,   2012.,   1861.,   1915.,   2005.,
         2009.,   1928.,   1978.,   1631.,   1820.,   1900.,    -50.,
         1921.,  -6550.,   1910.,  -5050.,  -7550.,  -6050.,   1770.,
         1863., -10450.,   1450.,   1250.,   1800.,  -2120.,  -8050.,
         1957.,   1923.,   2000.,  -2950.,    850.,  -2670.,   1000.,
          650.,   1256.,   1810.,    500.,  -1200.,   1253.,  -5350.,
         2019.,   2007.,   2022.,   1995.,   1793.,   2004.,    400.,
        -5060.,   1446.,   1981.,    260.,   1360.,  -4360.,   2006.,
         2014.,   1932.,   2017.,   1936.,   1847.,   1854.,   1979.,
         1946.,   2003.,   1866.,    440.,   1911.,   1550.,   1660.,
         1972.,   19

In [55]:
# Extract column needed for the basic NDIS database to compile with other geohazards dataset.
# Volcano Number --> HazardID, longitude, latitude, HazardType (in numerical coded added after extraction). Volcano is 1.
vo_df = volc[['VolcanoNumber','LatitudeDecimal','LongitudeDecimal']].copy()
vo_df.rename(columns = {'VolcanoNumber':'HazardID','LatitudeDecimal':'latitude','LongitudeDecimal':'longitude'}, inplace = True)
vo_df['HazardType'] = "Volcano"
vo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   HazardID    1324 non-null   int64  
 1   latitude    1324 non-null   float64
 2   longitude   1324 non-null   float64
 3   HazardType  1324 non-null   object 
dtypes: float64(2), int64(1), object(1)
memory usage: 41.5+ KB


In [47]:
len(volc[~volc["VolcanoNumber"].isin(volcano_merged["HazardID"])])

234

In [22]:
# Ensure correct data types
volcano_df["Volcano Number"] = volcano_df["Volcano Number"].astype(int)
merged_df_final["HazardID"] = merged_df_final["HazardID"].astype(int)

# Filter only volcano rows from merged_df_final
volcano_merged = merged_df_final[merged_df_final["HazardType"] == "Volcano"].copy()

# Find Volcano Numbers not in merged_df_final HazardIDs
volcano_not_in_merged = volcano_df[~volcano_df["Volcano Number"].isin(volcano_merged["HazardID"])]

# Find HazardIDs not in volcano_df
hazardid_not_in_volcano = volcano_merged[~volcano_merged["HazardID"].isin(volcano_df["Volcano Number"])]

# Output results
print("Volcano Numbers not in merged_df_final:")
print(len(volcano_not_in_merged["Volcano Number"].unique()))

print("\nHazardIDs not in volcano_df:")
print(len(hazardid_not_in_volcano["HazardID"].unique()))

Volcano Numbers not in merged_df_final:
203

HazardIDs not in volcano_df:
354


In [63]:
volc2_sdf.NAME.unique()

<StringArray>
[                     'Etna',                  'Karthala',
             'Ritter Island',                    'Rabaul',
                'Nyiragongo',                  'Vesuvius',
                    'Ischia',                    'Alayta',
                 'Stromboli',                  'Okataina',
 ...
                   'Kadovar',            'Billy Mitchell',
             'Sarychev Peak',                    'Ulawun',
          'Cendres, Ile des',                'Sumisujima',
                     'Dempo', "Hunga Tonga-Hunga Ha'apai",
                 'Reykjanes',                      'Agua']
Length: 273, dtype: string

In [65]:
# Normalize both name fields (strip, lowercase)
volcano_df["Volcano Name Clean"] = volcano_df["Volcano Name"].str.strip().str.lower()
volc2_sdf["NAME Clean"] = volc2_sdf["NAME"].str.strip().str.lower()

# Get all unique cleaned names from both
v1_names = set(volcano_df["Volcano Name Clean"].dropna().unique())
v2_names = set(volc2_sdf["NAME Clean"].dropna().unique())

# Intersect to find exact matches
matched_names = v1_names.intersection(v2_names)

# Show matched name samples
print(f"🔍 Found {len(matched_names)} name matches:")
print(sorted(list(matched_names))[:20])

🔍 Found 266 name matches:
['adatarayama', 'agua', 'agung', 'aira', 'akagisan', 'akita-yakeyama', 'alaid', 'alayta', 'ambae', 'ambalatungan group', 'ambrym', 'aniakchak', 'aogashima', 'apoyeque', 'ararat', 'arenal', 'arhab, harra of', 'asamayama', 'askja', 'asosan']


In [77]:
# Check column dtypes
print(volc2_clean[["LATITUDE", "LONGITUDE"]].dtypes)

# Find non-numeric rows
bad_rows = volc2_clean[
    ~volc2_clean["LATITUDE"].apply(lambda x: isinstance(x, (int, float))) |
    ~volc2_clean["LONGITUDE"].apply(lambda x: isinstance(x, (int, float)))
]

print(f"🚨 Found {len(bad_rows)} bad LAT/LON rows")
print(bad_rows[["LATITUDE", "LONGITUDE"]].head())


LATITUDE     Float64
LONGITUDE    Float64
dtype: object
🚨 Found 0 bad LAT/LON rows
Empty DataFrame
Columns: [LATITUDE, LONGITUDE]
Index: []


In [109]:
# Load
# Use known item ID
item_id2 = "e991eccebfa5425e9180d884af76f16a"  # Item ID
volc2 = gis.content.get(item_id2)

# Inspect the item
print(f"Title: {volc2.title}")
print(f"Type: {volc2.type}")

# Access the first layer in the feature service
v2_layer = volc2.layers[0]

# Load it into a Spatially Enabled DataFrame (SeDF)
volc2_sdf = v2_layer.query(where="1=1").sdf
volc2_sdf.info()

Title: Significant Volcanic Events
Type: Feature Service
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 60 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   YEAR                           892 non-null    Int32   
 1   MO                             759 non-null    Int32   
 2   DAY                            700 non-null    Int32   
 3   VEI                            712 non-null    Int32   
 4   VOL_ID                         892 non-null    Float64 
 5   FATALITIES                     451 non-null    Int32   
 6   ASSOC_EQ                       80 non-null     string  
 7   ASSOC_TSU                      178 non-null    string  
 8   COMMENTS                       876 non-null    string  
 9   DAMAGE_MILLIONS_DOLLARS        25 non-null     Float64 
 10  DAMAGE_AMOUNT_ORDER            249 non-null    Int32   
 11  DAMAGE_DESCRIPTION             249 non-n

In [53]:
len(volc2_sdf[volc2_sdf["LONGITUDE"].isin(volcano_merged["longitude"])])

686

In [34]:
volc2_sdf.DAMAGE_MILLIONS_DOLLARS_TOTAL.unique()

<FloatingArray>
[   <NA>,    19.0,    86.0,    15.0,     2.0,  2000.0,     4.0,     3.3,
     3.0,     5.0,   600.0,     2.5,    14.5,   100.0,     2.4,   370.0,
   3.564,   120.0,     1.0,   250.0,   115.0,    14.0,    67.0,   325.0,
   924.0,    20.0, 135.368,    15.5,     2.2]
Length: 29, dtype: Float64

In [ ]:
len(volc2_sdf[volc2_sdf["COMMENTS"].isin(volcano_merged["HazardID"])])

In [59]:
# Step 1: Clean the VEI field (drop missing VEI values)
vei_df = volcano_df.dropna(subset=["VEI"])

# Step 2: Aggregate by Volcano Number
vei_stats = vei_df.groupby("Volcano Number").agg(
    vei_max=("VEI", "max"),
    event_count=("VEI", "count")
).reset_index()

# Step 3: Merge with trimmed merged_df_final (only Volcano rows)
vei_merged = merged_trimmed[merged_trimmed["HazardType"] == "Volcano"].copy()
vei_merged = vei_merged.merge(vei_stats, left_on="HazardID", right_on="Volcano Number", how="left").drop(columns=["Volcano Number"])
vei_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090 entries, 0 to 1089
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   HazardID               1090 non-null   int32  
 1   latitude               1090 non-null   float64
 2   longitude              1090 non-null   float64
 3   HazardType             1090 non-null   object 
 4   distance               1044 non-null   float64
 5   pop                    1044 non-null   float64
 6   intensity              1044 non-null   float64
 7   economic_loss_million  1044 non-null   float64
 8   vei_max                575 non-null    float64
 9   event_count            575 non-null    float64
dtypes: float64(8), int32(1), object(1)
memory usage: 81.0+ KB


In [101]:
vei_stats.event_count.unique()

array([ 14,  60,   1,  22,  28, 122,   7,  12,   4,   5,   2,   3,   9,
         6,  45,  18,  30, 192,  40,  25,  77,  62,  17,  27,  11,  34,
        15,  39,   8,  29,  46,  24,  36,  53,  67,  38,  51,  19,  23,
        43,  81,  64,  68,  42,  72,  10,  21,  20,  13,  41,  31,  57,
        61,  26,  80, 180,  32, 128,  73,  44,  56, 105, 102,  48,  16,
       111,  33,  49,  54,  35, 152], dtype=int64)

In [80]:
# Prepare volcano_df: clean and drop missing coordinates
volcano_df_clean = volcano_df.dropna(subset=["Latitude", "Longitude"]).copy()
volcano_df_clean["Volcano Number"] = volcano_df_clean["Volcano Number"].astype(int)

volcano_df_clean = volcano_df.dropna(subset=["Latitude", "Longitude"]).copy()
volcano_df_clean = volcano_df_clean[(volcano_df_clean["Latitude"].apply(lambda x: isinstance(x, (int, float)))) &
                                    (volcano_df_clean["Longitude"].apply(lambda x: isinstance(x, (int, float))))]


# Prepare volc2_sdf: drop missing lat/lon and damage values
volc2_clean = volc2_sdf.dropna(subset=["LATITUDE", "LONGITUDE"]).copy()

# Ensure float type and no NaN
volc2_clean = volc2_sdf.dropna(subset=["LATITUDE", "LONGITUDE"]).copy()
volc2_clean = volc2_clean[(volc2_clean["LATITUDE"].apply(lambda x: isinstance(x, (int, float)))) &
                          (volc2_clean["LONGITUDE"].apply(lambda x: isinstance(x, (int, float))))]


# Degrees to radians manually (no ufunc)
DEG2RAD = np.pi / 180.0

v1_coords = volcano_df_clean[["Latitude", "Longitude"]].astype("float64").values * DEG2RAD
v2_coords = volc2_clean[["LATITUDE", "LONGITUDE"]].astype("float64").values * DEG2RAD

# Force native float64 before using numpy
volc2_clean["LATITUDE"] = volc2_clean["LATITUDE"].astype("float64")
volc2_clean["LONGITUDE"] = volc2_clean["LONGITUDE"].astype("float64")

# Force dtype to float64 before np.radians
v2_coords = np.radians(volc2_clean[["LATITUDE", "LONGITUDE"]].astype("float64").values)

# Build BallTree and query nearest neighbors
tree = BallTree(v2_coords, metric="haversine")
distances, indices = tree.query(v1_coords, k=1)

# Convert to kilometers
distances_km = distances.flatten() * 6371

# Attach distance and matched index to volcano_df_clean
volcano_df_clean["nearest_volc2_index"] = indices.flatten()
volcano_df_clean["nearest_distance_km"] = distances_km

# Count how many are within 50 km
within_50km = (volcano_df_clean["nearest_distance_km"] <= 50).sum()
print(f"🌋 Volcanoes matched within 50 km: {within_50km} of {len(volcano_df_clean)}")

🌋 Volcanoes matched within 50 km: 8475 of 11282


In [87]:
print(volcano_df_clean.columns.tolist())

['Volcano Number', 'Volcano Name', 'Eruption Number', 'Eruption Category', 'Area of Activity', 'VEI', 'VEI Modifier', 'Start Year Modifier', 'Start Year', 'Start Year Uncertainty', 'Start Month', 'Start Day Modifier', 'Start Day', 'Start Day Uncertainty', 'Evidence Method (dating)', 'End Year Modifier', 'End Year', 'End Year Uncertainty', 'End Month', 'End Day Modifier', 'End Day', 'End Day Uncertainty', 'Latitude', 'Longitude', 'Volcano Name Clean', 'nearest_volc2_index', 'nearest_distance_km']


In [88]:
volcano_df_clean["damage_million_est"] = volc2_clean.iloc[
    volcano_df_clean["nearest_volc2_index"].values
]["DAMAGE_MILLIONS_DOLLARS_TOTAL"].values

In [91]:
# Step 1: Filter only volcanoes matched within 50 km
volcano_damage = volcano_df_clean.loc[
    volcano_df_clean["nearest_distance_km"] <= 50,
    ["Volcano Number", "damage_million_est"]
].copy()

# Step 2: Aggregate to ensure only one row per Volcano Number
volcano_damage_cleaned = volcano_damage.groupby(
    "Volcano Number", as_index=False
)["damage_million_est"].max()

# Step 3: Merge with vei_merged
vei_merged_updated = vei_merged.merge(
    volcano_damage_cleaned,
    left_on="HazardID",
    right_on="Volcano Number",
    how="left"
).drop(columns=["Volcano Number"])
vei_merged_updated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090 entries, 0 to 1089
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   HazardID               1090 non-null   int32  
 1   latitude               1090 non-null   float64
 2   longitude              1090 non-null   float64
 3   HazardType             1090 non-null   object 
 4   distance               1044 non-null   float64
 5   pop                    1044 non-null   float64
 6   intensity              1044 non-null   float64
 7   economic_loss_million  1044 non-null   float64
 8   vei_max                575 non-null    float64
 9   event_count            575 non-null    float64
 10  damage_million_est     14 non-null     Float64
dtypes: Float64(1), float64(8), int32(1), object(1)
memory usage: 90.6+ KB


In [90]:
len(vei_merged_updated.HazardID.unique())

1090

In [92]:
vei_merged_updated

,HazardID,latitude,longitude,HazardType,distance,pop,intensity,economic_loss_million,vei_max,event_count,damage_million_est
0,210010,50.170,6.850,Volcano,34.462092,1.346558e+05,2.0,2.82,NaN,NaN,<NA>
1,210020,45.775,2.970,Volcano,1068.063073,4.145101e+05,3.0,9.86,NaN,NaN,<NA>
2,210030,42.170,2.530,Volcano,1798.388196,1.461354e+05,1.0,0.51,NaN,NaN,<NA>
3,210040,38.870,-4.020,Volcano,547.012635,1.739199e+05,2.0,2.62,NaN,NaN,<NA>
4,211004,41.730,12.700,Volcano,183.805721,2.411999e+06,1.0,0.99,NaN,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...
1085,390110,-56.712,-27.176,Volcano,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
1086,390120,-56.656,-28.140,Volcano,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
1087,390847,-62.020,-57.670,Volcano,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
1088,390829,-64.150,-57.750,Volcano,NaN,NaN,NaN,NaN,NaN,NaN,<NA>


In [102]:
vei_merged_updated.to_csv(r"D:\NDIS_Database\13_NDIS_Display\Page2Layers\Volcano_stat.csv", index=False)

In [119]:
vei_merged_updated = pd.read_csv(r"D:\NDIS_Database\13_NDIS_Display\Page2Layers\Volcano_stat.csv")
vei_merged_updated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090 entries, 0 to 1089
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   HazardID               1090 non-null   int64  
 1   latitude               1090 non-null   float64
 2   longitude              1090 non-null   float64
 3   HazardType             1090 non-null   object 
 4   distance               1044 non-null   float64
 5   pop                    1044 non-null   float64
 6   intensity              1044 non-null   float64
 7   economic_loss_million  1044 non-null   float64
 8   vei_max                575 non-null    float64
 9   event_count            575 non-null    float64
 10  damage_million_est     14 non-null     float64
dtypes: float64(9), int64(1), object(1)
memory usage: 93.8+ KB


In [8]:
vei_merged_updated.vei_max.unique()

array([nan,  5.,  3.,  4.,  1.,  7.,  0.,  2.,  6.])

In [121]:
# Select only the desired columns from merged_df_final
vei_merged_updated = vei_merged_updated[[
    "HazardID",
    "latitude",
    "longitude",
    "distance",
    "pop",
    "vei_max",
    "event_count",
    "damage_million_est"
]].copy()
vei_merged_updated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090 entries, 0 to 1089
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   HazardID            1090 non-null   int64  
 1   latitude            1090 non-null   float64
 2   longitude           1090 non-null   float64
 3   distance            1044 non-null   float64
 4   pop                 1044 non-null   float64
 5   vei_max             575 non-null    float64
 6   event_count         575 non-null    float64
 7   damage_million_est  14 non-null     float64
dtypes: float64(7), int64(1)
memory usage: 68.3 KB


In [127]:
# Step 1: Extract relevant fields and rename for merge clarity
last_eruption_df = volc[["VolcanoNumber", "LastEruption", "VolcanoName"]].copy()

# Step 2: Merge into vei_merged_updated
vei_merged_final = vei_merged_updated.merge(
    last_eruption_df,
    left_on="HazardID",
    right_on="VolcanoNumber",
    how="left"
)

# Step 3: Drop the duplicate key if not needed
vei_merged_final.drop(columns=["VolcanoNumber"], inplace=True)

In [128]:
# Define a formatter function
def format_eruption_year(year):
    if pd.isna(year):
        return None
    elif year < 0:
        return f"{int(abs(year))} BCE"
    else:
        return f"{int(year)} CE"

# Apply the formatter to create a label column
vei_merged_final["LastEruptionLabel"] = vei_merged_final["LastEruption"].apply(format_eruption_year)
vei_merged_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090 entries, 0 to 1089
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   HazardID            1090 non-null   int64  
 1   latitude            1090 non-null   float64
 2   longitude           1090 non-null   float64
 3   distance            1044 non-null   float64
 4   pop                 1044 non-null   float64
 5   vei_max             575 non-null    float64
 6   event_count         575 non-null    float64
 7   damage_million_est  14 non-null     float64
 8   LastEruption        688 non-null    float64
 9   VolcanoName         1090 non-null   object 
 10  LastEruptionLabel   688 non-null    object 
dtypes: float64(8), int64(1), object(2)
memory usage: 93.8+ KB


In [129]:
vei_merged_final.drop(columns=["LastEruption"], inplace=True)

In [132]:
vei_merged_final.to_csv(r"D:\NDIS_Database\13_NDIS_Display\Page3\volcano_stat.csv", index=False)

# Landslide

In [157]:
# Read the shapefile from local disk
landslide_df = gpd.read_file(r"D:/NDIS_Database/04_Landslide/nasa_coolr_reports_point.shp")
landslide_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   src_name    14718 non-null  object  
 1   src_link    13879 non-null  object  
 2   ev_date     13087 non-null  object  
 3   ev_time     6555 non-null   object  
 4   ev_title    14026 non-null  object  
 5   ev_desc     12176 non-null  object  
 6   loc_desc    13195 non-null  object  
 7   loc_acc     14693 non-null  object  
 8   ls_cat      14720 non-null  object  
 9   ls_trig     14720 non-null  object  
 10  ls_size     14719 non-null  object  
 11  ls_setting  14713 non-null  object  
 12  fatalities  14723 non-null  int64   
 13  injuries    14723 non-null  int64   
 14  storm_name  666 non-null    object  
 15  photo_link  2228 non-null   object  
 16  comments    2560 non-null   object  
 17  ev_imp_src  14723 non-null  object  
 18  ev_imp_id   11689 non-null  object  
 

In [143]:
landslide_df.ls_cat

0          landslide
1           mudslide
2          landslide
3          landslide
4          landslide
            ...     
14718      rock_fall
14719      rock_fall
14720    debris_flow
14721      rock_fall
14722    debris_flow
Name: ls_cat, Length: 14723, dtype: object

In [161]:
# Merge data to main landslide in database
# 1. Ensure keys match (rename 'ev_id' to 'HazardID' if needed)
landslide_df = landslide_df.rename(columns={"ev_id": "HazardID"})

# Keep only one row per HazardID (e.g., first)
#landslide_df_dedup = landslide_df.drop_duplicates(subset="HazardID", keep="first")

# 2. Select only the fields you want to bring over
fields_to_merge = ["HazardID", "ls_cat", "ls_size", "ev_date", "ev_time", "ls_setting", "ls_size", "fatalities", "injuries"]

# # 3. Merge into filtered main_db landslide records
landslide_db = merged_trimmed[merged_trimmed["HazardType"] == "Landslide"].copy()

# 4. Merge attributes
landslide_db = landslide_db.merge(
    landslide_df[fields_to_merge],
    on="HazardID",
    how="left"
)
landslide_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14437 entries, 0 to 14436
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   HazardID               14437 non-null  int64  
 1   latitude               14437 non-null  float64
 2   longitude              14437 non-null  float64
 3   HazardType             14437 non-null  object 
 4   distance               14080 non-null  float64
 5   pop                    14080 non-null  float64
 6   intensity              14080 non-null  float64
 7   economic_loss_million  14080 non-null  float64
 8   ls_cat                 14434 non-null  object 
 9   ls_size                14434 non-null  object 
 10  ev_date                12862 non-null  object 
 11  ev_time                6445 non-null   object 
 12  ls_setting             14427 non-null  object 
 13  ls_size                14434 non-null  object 
 14  fatalities             14437 non-null  int64  
 15  in

In [162]:
# Drop unwanted fields from landslide
landslide_db = landslide_db.drop(
    columns=["HazardType", "intensity", "economic_loss_million"]
)
landslide_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14437 entries, 0 to 14436
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   HazardID    14437 non-null  int64  
 1   latitude    14437 non-null  float64
 2   longitude   14437 non-null  float64
 3   distance    14080 non-null  float64
 4   pop         14080 non-null  float64
 5   ls_cat      14434 non-null  object 
 6   ls_size     14434 non-null  object 
 7   ev_date     12862 non-null  object 
 8   ev_time     6445 non-null   object 
 9   ls_setting  14427 non-null  object 
 10  ls_size     14434 non-null  object 
 11  fatalities  14437 non-null  int64  
 12  injuries    14437 non-null  int64  
dtypes: float64(4), int64(3), object(6)
memory usage: 1.4+ MB


In [163]:
landslide_db.to_csv(r"D:\NDIS_Database\13_NDIS_Display\landslide_stat.csv", index=False)

In [165]:
# Set environment
arcpy.env.overwriteOutput = True
arcpy.env.workspace = r"D:\NDIS_Database\13_NDIS_Display\Page2Layers"

# Save DataFrame to CSV
temp_csv = r"D:\NDIS_Database\13_NDIS_Display\Page2Layers\landslide_temp.csv"
landslide_db.to_csv(temp_csv, index=False)

# Parameters for MakeXYEventLayer
x_field = "longitude"
y_field = "latitude"
xy_layer = "Landslide_Layer"
spatial_ref = arcpy.SpatialReference(4326)  # WGS 1984

# Create event layer
arcpy.MakeXYEventLayer_management(
    temp_csv,
    x_field,
    y_field,
    xy_layer,
    spatial_ref
)

# Export to feature class (inside current workspace)
output_fc = "Landslide_Output"
arcpy.CopyFeatures_management(xy_layer, output_fc)

<Result 'D:\\NDIS_Database\\13_NDIS_Display\\Page2Layers\\Landslide_Output.shp'>

# Tsunami

In [3]:
# Load
# Use known item ID
item_id3 = "5a44c3d4d465498993120b70ab568876"  # Item ID
tsun = gis.content.get(item_id3)

# Inspect the item
print(f"Title: {tsun.title}")
print(f"Type: {tsun.type}")

# Access the first layer in the feature service
tsun_layer = tsun.layers[0]

# Load it into a Spatially Enabled DataFrame (SeDF)
tsun_sdf = tsun_layer.query(where="1=1").sdf
tsun_sdf.info()

Title: Historical Tsunami Events
Type: Feature Service
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2625 entries, 0 to 2624
Data columns (total 76 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   UNIQUE_ID                      2625 non-null   Int64         
 1   YEAR                           2625 non-null   Int32         
 2   MONTH                          2514 non-null   Int32         
 3   DAY                            2430 non-null   Int32         
 4   HOUR                           1554 non-null   Int32         
 5   MINUTE                         1471 non-null   Int32         
 6   SECOND                         1047 non-null   Float64       
 7   DATE_STRING                    2625 non-null   string        
 8   EVENT_DATE                     2596 non-null   datetime64[us]
 9   LATITUDE                       2625 non-null   Float64       
 10  LONGITUDE                    

In [7]:
tsun_merged = merged_trimmed[merged_trimmed["HazardType"] == "Tsunami"].copy()
tsun_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 738 entries, 15108 to 1817011
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   HazardID    738 non-null    int64  
 1   latitude    738 non-null    float64
 2   longitude   738 non-null    float64
 3   HazardType  738 non-null    object 
 4   distance    369 non-null    float64
 5   pop         369 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 40.4+ KB


In [8]:
# Merge tsun_sdf with merged_trimmed using Unique_ID (mapped from HazardID)
# Only keep necessary columns from merged_trimmed
merged_cols = ["HazardID", "distance", "pop"]
merged_subset = merged_trimmed[merged_cols]

# Merge where HazardID in merged_trimmed == UNIQUE_ID in tsun_sdf
tsun_keep_columns = [
    "UNIQUE_ID", "LATITUDE", "LONGITUDE", "LOCATION_NAME", "CAUSE",
    "TS_MT_ABE", "TS_MT_II", "TS_INTENSITY", "EVENT_DATE",
    "DAMAGE_MILLIONS_DOLLARS_TOTAL", "HOUSES_DESTROYED_TOTAL",
    "DEATHS_TOTAL", "INJURIES_TOTAL", "MISSING_TOTAL"
]

tsun_filtered = tsun_sdf[tsun_keep_columns]

# Perform the merge
tsun_merged = tsun_filtered.merge(
    merged_subset,
    left_on="UNIQUE_ID",
    right_on="HazardID",
    how="left"
)

# Drop the duplicate HazardID column after merge
tsun_merged.drop(columns=["HazardID"], inplace=True)
tsun_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2625 entries, 0 to 2624
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   UNIQUE_ID                      2625 non-null   Int64         
 1   LATITUDE                       2625 non-null   Float64       
 2   LONGITUDE                      2625 non-null   Float64       
 3   LOCATION_NAME                  2624 non-null   string        
 4   CAUSE                          2623 non-null   string        
 5   TS_MT_ABE                      2 non-null      Float64       
 6   TS_MT_II                       711 non-null    Float64       
 7   TS_INTENSITY                   1176 non-null   Float64       
 8   EVENT_DATE                     2596 non-null   datetime64[us]
 9   DAMAGE_MILLIONS_DOLLARS_TOTAL  172 non-null    Float64       
 10  HOUSES_DESTROYED_TOTAL         266 non-null    Int32         
 11  DEATHS_TOTAL     

In [10]:
tsun_merged.to_csv(r"D:\NDIS_Database\13_NDIS_Display\Page3\tsunami_stat.csv", index=False)

# Fault

In [5]:
# Input feature class
input_fc = r"D:\NDIS_Database\05_Fault\gem-global-active-faults-master\gem-global-active-faults-master\geopackage\gem_active_faults_harmonized.gpkg\main.gem_active_faults_harmonized"

# Output feature class (trimmed copy)
output_fc = r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\fault_lines_trimmed"

# List of fields to KEEP (make sure to include 'Shape@' or geometry field!)
fields_to_keep = [
    "Id", "name", "slip_type", "lat_movement", "geom", "geom_Length"  # example
]

# Create a new feature class with only those fields
arcpy.management.CopyFeatures(input_fc, output_fc)

# Get all fields in original
all_fields = [f.name for f in arcpy.ListFields(output_fc) if f.name not in ("OBJECTID", "Shape")]

# Calculate which to delete
fields_to_delete = [f for f in all_fields if f not in fields_to_keep]

# Delete unneeded fields
arcpy.management.DeleteField(output_fc, fields_to_delete)

print("Trimmed feature class created at:", output_fc)

Trimmed feature class created at: D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\fault_lines_trimmed


In [63]:
# Example path to the fault line feature class
gdb_path = r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb"
layer_name = "fault_lines_trimmed"  # Replace with actual name

# Read into Spatially Enabled DataFrame
fault_sdf = pd.DataFrame.spatial.from_featureclass(f"{gdb_path}\\{layer_name}")
fault_sdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13696 entries, 0 to 13695
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   OBJECTID   13696 non-null  Int64   
 1   name       4770 non-null   string  
 2   slip_type  13377 non-null  string  
 3   SHAPE      13696 non-null  geometry
dtypes: Int64(1), geometry(1), string(2)
memory usage: 441.5 KB


In [68]:
fault_sdf

,OBJECTID,name,slip_type,SHAPE
0,1,Mount Diablo Thrust,Reverse,"{""paths"": [[[-121.82289999999995, 37.730100000..."
1,2,Ortigalita (South),Dextral,"{""paths"": [[[-121.14289999999994, 37.108100000..."
2,3,Monterey Bay-Tularcitos,Dextral,"{""paths"": [[[-121.50308999999999, 36.357590000..."
3,4,Mendocino,Dextral,"{""paths"": [[[-124.34335999999996, 40.206670000..."
4,5,Table Bluff,Reverse,"{""paths"": [[[-124.17976599999997, 40.647781000..."
...,...,...,...,...
13691,13692,Panga-4,Normal,"{""paths"": [[[34.62431000000004, -16.2497699999..."
13692,13693,Tsikulamowa,Normal,"{""paths"": [[[34.82039000000003, -14.9235399999..."
13693,13694,Nsanje,Normal,"{""paths"": [[[35.24663000000004, -16.8851999999..."
13694,13695,Bilila-Mtakataka,Normal,"{""paths"": [[[34.506200000000035, -14.267529999..."


In [65]:
fault_pts = merged_trimmed[merged_trimmed["HazardType"] == "Fault"].copy()
fault_pts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 172823 entries, 1671180 to 1931884
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   HazardID               172823 non-null  int64  
 1   latitude               172823 non-null  float64
 2   longitude              172823 non-null  float64
 3   HazardType             172823 non-null  object 
 4   distance               143519 non-null  float64
 5   pop                    143519 non-null  float64
 6   intensity              143519 non-null  float64
 7   economic_loss_million  143519 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 11.9+ MB


In [70]:
# Read the shapefile from local disk
fault_df = gpd.read_file(r"D:/NDIS_Database/05_Fault/fault_points.shp")
fault_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 190565 entries, 0 to 190564
Data columns (total 28 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   ORIG_FID    190565 non-null  int64   
 1   average_di  87654 non-null   object  
 2   average_ra  55884 non-null   object  
 3   catalog_id  190565 non-null  object  
 4   catalog_na  190565 non-null  object  
 5   dip_dir     39068 non-null   object  
 6   lower_seis  57659 non-null   object  
 7   name        48712 non-null   object  
 8   net_slip_r  123773 non-null  object  
 9   slip_type   187189 non-null  object  
 10  upper_seis  65947 non-null   object  
 11  reference   28250 non-null   object  
 12  epistemic_  82475 non-null   object  
 13  accuracy    21751 non-null   object  
 14  activity_c  79082 non-null   object  
 15  fs_name     22773 non-null   object  
 16  last_movem  9213 non-null    object  
 17  downthrown  12593 non-null   object  
 18  vert_sep_r  1598

In [73]:
fault_df.ORIG_FID.unique()

array([    0,     1,     2, ..., 13693, 13694, 13695], dtype=int64)

In [67]:
fault_pts

,HazardID,latitude,longitude,HazardType,distance,pop,intensity,economic_loss_million
1671180,1771000034,17.287225,93.876733,Fault,70303.019280,0.0,1.0,0.0
1671181,1771000101,0.590237,96.755289,Fault,83119.364773,0.0,1.0,0.0
1671182,1771000116,0.946438,96.549546,Fault,82169.630145,0.0,1.0,0.0
1671183,1771000364,-2.980043,98.909096,Fault,110734.680678,0.0,1.0,0.0
1671184,1771000440,-2.184676,98.294718,Fault,96578.361124,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...
1931880,1774711619,19.591019,-67.432301,Fault,NaN,NaN,NaN,NaN
1931881,1773140627,19.607422,-67.551839,Fault,NaN,NaN,NaN,NaN
1931882,1779432084,19.612876,-67.507908,Fault,NaN,NaN,NaN,NaN
1931883,1777859221,19.627115,-67.464477,Fault,NaN,NaN,NaN,NaN


In [76]:
# Group by ORIG_FID and compute min for distance and mean for pop
agg_vals = fault_pts.groupby("HazardID")[["distance", "pop"]].agg({
    "distance": "min",   # Closest road point
    "pop": "mean"        # Average population along fault
}).reset_index()
agg_vals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172823 entries, 0 to 172822
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   HazardID  172823 non-null  int64  
 1   distance  143519 non-null  float64
 2   pop       143519 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 4.0 MB


In [77]:
# Merge fault_df with fault_sdf using ORIG_FID ↔ OBJECTID
fault_poly_df = fault_df.merge(
    fault_sdf[["OBJECTID", "SHAPE"]],
    left_on="ORIG_FID",
    right_on="OBJECTID",
    how="left"
)

fault_poly_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 190565 entries, 0 to 190564
Data columns (total 30 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   ORIG_FID    190565 non-null  int64   
 1   average_di  87654 non-null   object  
 2   average_ra  55884 non-null   object  
 3   catalog_id  190565 non-null  object  
 4   catalog_na  190565 non-null  object  
 5   dip_dir     39068 non-null   object  
 6   lower_seis  57659 non-null   object  
 7   name        48712 non-null   object  
 8   net_slip_r  123773 non-null  object  
 9   slip_type   187189 non-null  object  
 10  upper_seis  65947 non-null   object  
 11  reference   28250 non-null   object  
 12  epistemic_  82475 non-null   object  
 13  accuracy    21751 non-null   object  
 14  activity_c  79082 non-null   object  
 15  fs_name     22773 non-null   object  
 16  last_movem  9213 non-null    object  
 17  downthrown  12593 non-null   object  
 18  vert_sep_r  1598

# Earthquake

In [93]:
eq_df = pd.read_csv(r"D:\NDIS_Database\02_Earthquake\eq_database_complete.csv") # Load earthquake data
eq_df.info()

C:/Users/Dell/AppData/Local/Temp/ArcGISProTemp14016/xpython_14016/959616331.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  eq_df = pd.read_csv(r"D:\NDIS_Database\02_Earthquake\eq_database_complete.csv") # Load earthquake data


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1719979 entries, 0 to 1719978
Data columns (total 32 columns):
 #   Column          Dtype  
---  ------          -----  
 0   eventID         int64  
 1   Agency          object 
 2   year            int64  
 3   month           int64  
 4   day             int64  
 5   hour            int64  
 6   minute          int64  
 7   second          float64
 8   timeError       float64
 9   longitude       float64
 10  latitude        float64
 11  SemiMajor90     object 
 12  SemiMinor90     object 
 13  ErrorStrike     float64
 14  depth           float64
 15  depthError      float64
 16  depthfix        object 
 17  magnitude       float64
 18  sigmaMagnitude  float64
 19  magnitudeType   object 
 20  catName         object 
 21  conv_method     object 
 22  agency_conv     object 
 23  orig_mag        float64
 24  orig_MagType    object 
 25  cluster_index   int64  
 26  cluster_flag    int64  
 27  dtime           float64
 28  completeness

In [99]:
eq_df

,eventID,Agency,year,month,day,hour,minute,second,timeError,longitude,...,agency_conv,orig_mag,orig_MagType,cluster_index,cluster_flag,dtime,completeness,year_comp,CompZone,crust_sub
0,210,AHEAD,1000,3,29,0,0,0.00,NaN,4.2370,...,Mw,3.7,Mw,0,0,1000.238356,0.0,NaN,FRAB,Crustal
1,360,AHEAD,1003,3,21,0,0,0.00,NaN,38.8000,...,Mw,6.2,Mw,0,0,1003.216438,0.0,NaN,ETUR,Crustal
2,600,AHEAD,1005,1,1,0,0,0.00,NaN,13.8310,...,Mw,5.2,Mw,0,0,1005.000000,0.0,NaN,MEDB,Crustal
3,500,AHEAD,1005,1,1,0,0,0.00,NaN,11.8790,...,Mw,5.2,Mw,0,0,1005.000000,0.0,NaN,CITA,Crustal
4,10080427180,AmbraseysandMelville,1008,4,27,18,0,0.00,NaN,47.4000,...,Ms_Mw,7.0,Ms,0,0,1008.321721,0.0,NaN,ZAGROS,Crustal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1719974,618589977,MDD_MDD,2020,6,25,3,51,12.70,NaN,-9.5980,...,Mb_MDD_Mw,3.1,Mb,0,0,2020.437904,1.0,1978.0,OFFP,Crustal
1719975,618589982,BUC_BUC,2020,6,25,5,42,41.89,NaN,23.1581,...,Ml_BUC_Mw,3.3,Ml_ROM_BUC,0,0,2020.437979,1.0,1977.0,SBAL,Crustal
1719976,618589983,MDD_MDD,2020,6,25,5,47,35.61,NaN,-4.6010,...,Mb_MDD_Mw,3.7,Mb,0,0,2020.438069,1.0,1973.0,BECA,Crustal
1719977,618589986,NEIC_NEIC,2020,6,25,6,8,12.00,NaN,-68.9540,...,M_NEIC_Mw,3.8,M,101991,0,2020.438157,1.0,1994.0,CARAB,DeepEvents


In [95]:
eq_df.magnitudeType.unique()

array(['Mw'], dtype=object)

In [96]:
eq_db = pd.read_csv(r"D:\NDIS_Database\02_Earthquake\eq_rf.csv") # Load earthquake data
eq_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1655703 entries, 0 to 1655702
Data columns (total 12 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   HazardID               1655703 non-null  int64  
 1   latitude               1655703 non-null  float64
 2   longitude              1655703 non-null  float64
 3   HazardType             1655703 non-null  int64  
 4   distance               1655703 non-null  float64
 5   geometry               1655703 non-null  object 
 6   pop                    1655703 non-null  float64
 7   magnitude              1636279 non-null  float64
 8   depth                  1614980 non-null  float64
 9   intensity              1636279 non-null  float64
 10  duration_minutes       1636279 non-null  float64
 11  economic_loss_million  1636279 non-null  float64
dtypes: float64(9), int64(2), object(1)
memory usage: 151.6+ MB


In [103]:
# Create unified datetime column
eq_df["time"] = pd.to_datetime(
    eq_df[["year", "month", "day", "hour", "minute", "second"]],
    errors="coerce"
)

# Select and copy needed columns from eq_df
eq_df_filtered = eq_df[["eventID", "orig_mag", "orig_MagType", "time"]].copy()

# Drop unwanted fields from eq_db
eq_db_cleaned = eq_db.drop(
    columns=["magnitude", "intensity", "duration_minutes", "economic_loss_million"]
)

# Merge: keep all from eq_df, bring matching values from eq_db
eq_merged = eq_df_filtered.merge(
    eq_db_cleaned,
    left_on="eventID",
    right_on="HazardID",
    how="left"
)

eq_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1719979 entries, 0 to 1719978
Data columns (total 12 columns):
 #   Column        Dtype         
---  ------        -----         
 0   eventID       int64         
 1   orig_mag      float64       
 2   orig_MagType  object        
 3   time          datetime64[ns]
 4   HazardID      float64       
 5   latitude      float64       
 6   longitude     float64       
 7   HazardType    float64       
 8   distance      float64       
 9   geometry      object        
 10  pop           float64       
 11  depth         float64       
dtypes: datetime64[ns](1), float64(8), int64(1), object(2)
memory usage: 157.5+ MB


In [105]:
# Drop unwanted fields from eq_db
eq_merged = eq_merged.drop(
    columns=["eventID", "HazardType"]
)

In [106]:
eq_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1719979 entries, 0 to 1719978
Data columns (total 10 columns):
 #   Column        Dtype         
---  ------        -----         
 0   orig_mag      float64       
 1   orig_MagType  object        
 2   time          datetime64[ns]
 3   HazardID      float64       
 4   latitude      float64       
 5   longitude     float64       
 6   distance      float64       
 7   geometry      object        
 8   pop           float64       
 9   depth         float64       
dtypes: datetime64[ns](1), float64(7), object(2)
memory usage: 131.2+ MB


In [108]:
eq_merged.to_csv(r"D:\NDIS_Database\13_NDIS_Display\Page3\eq_stat.csv", index=False)

# Nuclear Power Plant

In [53]:
# Read data
nuclear_stat = pd.read_csv(r"D:\NDIS_Database\13_NDIS_Display\Page3\nuclear_stat.csv")
nuclear_stat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1541 entries, 0 to 1540
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   GEM unit ID    1541 non-null   object 
 1   Project Name   1541 non-null   object 
 2   Longitude      1541 non-null   float64
 3   Latitude       1541 non-null   float64
 4   Capacity (MW)  1541 non-null   object 
 5   Country/Area   1541 non-null   object 
 6   Owner          1372 non-null   object 
 7   Status         1541 non-null   object 
 8   Reactor Type   1541 non-null   object 
 9   Wiki URL       1541 non-null   object 
dtypes: float64(2), object(8)
memory usage: 120.5+ KB


In [57]:
# Read data
nuclear_clean = pd.read_csv(r"D:\NDIS_Database\cleaned_nuclear_df.csv")
nuclear_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1541 entries, 0 to 1540
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   HazardID               1541 non-null   int64  
 1   latitude               1541 non-null   float64
 2   longitude              1541 non-null   float64
 3   HazardType             1541 non-null   object 
 4   intensity              1541 non-null   float64
 5   duration_minutes       1541 non-null   float64
 6   economic_loss_million  1541 non-null   float64
 7   travel_time            0 non-null      float64
 8   monitor_time           1541 non-null   float64
 9   cpm_total_time         0 non-null      float64
 10  HazardStage            1541 non-null   object 
 11  pop                    1541 non-null   float64
 12  distance               1541 non-null   float64
dtypes: float64(10), int64(1), object(2)
memory usage: 156.6+ KB


In [56]:
nuclear_merged.HazardID.unique()

array([2081454870, 2081454750, 2081455014, ..., 2081455862, 2081455863,
       2081455864], dtype=int64)

In [58]:
# Step 1: Back-calculate GEM unit ID from HazardID in nuclear_clean
nuclear_clean = nuclear_clean.copy()
nuclear_clean["GEM_unit_numeric"] = nuclear_clean["HazardID"].astype(str).str[3:]  # Remove "111" prefix

# Step 2: Merge using GEM numeric ID
nuclear_stat = nuclear_stat.copy()
nuclear_stat["GEM_unit_numeric"] = nuclear_stat["GEM unit ID"].astype(str).str.extract(r"(\d+)$")[0]

# Step 3: Merge on the extracted numeric ID
nuclear_merged = nuclear_clean.merge(
    nuclear_stat,
    on="GEM_unit_numeric",
    how="left"
)

# Step 4: Clean up
nuclear_merged.drop(columns=["GEM_unit_numeric"], inplace=True)

In [59]:
nuclear_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1541 entries, 0 to 1540
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   HazardID               1541 non-null   int64  
 1   latitude               1541 non-null   float64
 2   longitude              1541 non-null   float64
 3   HazardType             1541 non-null   object 
 4   intensity              1541 non-null   float64
 5   duration_minutes       1541 non-null   float64
 6   economic_loss_million  1541 non-null   float64
 7   travel_time            0 non-null      float64
 8   monitor_time           1541 non-null   float64
 9   cpm_total_time         0 non-null      float64
 10  HazardStage            1541 non-null   object 
 11  pop                    1541 non-null   float64
 12  distance               1541 non-null   float64
 13  GEM unit ID            1541 non-null   object 
 14  Project Name           1541 non-null   object 
 15  Long

In [61]:
# Step 5: Keep only required fields
final_nuclear_df = nuclear_merged[[
    "GEM unit ID", "Project Name", "Longitude", "Latitude",
    "Capacity (MW)", "Country/Area", "Owner", "Status",
    "Reactor Type", "Wiki URL", "pop", "distance"
]].copy()

final_nuclear_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1541 entries, 0 to 1540
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   GEM unit ID    1541 non-null   object 
 1   Project Name   1541 non-null   object 
 2   Longitude      1541 non-null   float64
 3   Latitude       1541 non-null   float64
 4   Capacity (MW)  1541 non-null   object 
 5   Country/Area   1541 non-null   object 
 6   Owner          1372 non-null   object 
 7   Status         1541 non-null   object 
 8   Reactor Type   1541 non-null   object 
 9   Wiki URL       1541 non-null   object 
 10  pop            1541 non-null   float64
 11  distance       1541 non-null   float64
dtypes: float64(4), object(8)
memory usage: 144.6+ KB


In [62]:
final_nuclear_df.to_csv(r"D:\NDIS_Database\13_NDIS_Display\Page3\nuclear_stat.csv", index=False)